In [5]:
import os
import cv2
from ultralytics import YOLO
import pandas as pd

# Load YOLOv8 Pose model
model = YOLO("yolo11s-pose.pt")

# Open video file
cap = cv2.VideoCapture('normal.mp4')

# Get frame count and fps
frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)
seconds = round(frames / fps)

frame_total = 1000
i = 0
a = 0
all_data = []

while cap.isOpened():
    cap.set(cv2.CAP_PROP_POS_MSEC, (i * ((seconds / frame_total) * 1000)))
    ret, frame = cap.read()

    if not ret:
        break

    # Save full frame to images/
    full_image_path = f'images/img_{i}.jpg'
    cv2.imwrite(full_image_path, frame)

    # Pose detection
    results = model(frame, verbose=False)

    for r in results:
        bound_box = r.boxes.xyxy
        conf = r.boxes.conf.tolist()
        keypoints = r.keypoints.xyn.tolist()

        for index, box in enumerate(bound_box):
            if conf[index] > 0.75:
                x1, y1, x2, y2 = map(int, box.tolist())
                cropped_person = frame[y1:y2, x1:x2]

                cropped_image_path = f'images1/person_nn_{a}.jpg'
                cv2.imwrite(cropped_image_path, cropped_person)

                # Save keypoints
                data = {'image_name': f'person_nn_{a}.jpg'}
                for j, (x, y) in enumerate(keypoints[index]):
                    data[f'x{j}'] = x
                    data[f'y{j}'] = y

                all_data.append(data)
                a += 1

    i += 1

cap.release()
cv2.destroyAllWindows()

# Save to CSV
df = pd.DataFrame(all_data)
csv_path = 'nkeypoint.csv'
df.to_csv(csv_path, index=False)

print(f"✅ Finished! Frames: {i}, Crops: {a}")
print("🖼️ Saved to 'images/' and 'images1/'")
print(f"📄 Keypoints CSV saved as: {csv_path}")

✅ Finished! Frames: 1007, Crops: 1500
🖼️ Saved to 'images/' and 'images1/'
📄 Keypoints CSV saved as: nkeypoint.csv
